**This notebook is to finetune ResMemNet on US 10K Faces dataset and then use the model to predict memorability scores of CelebA dataset. Also used in the final testing of the model to verify change in memorability.**


###Training ResMemNet

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
from skimage import io,transform
import torch
from tqdm import tqdm
from torch import nn
import torch.nn.functional as F
import os
import time
from torch.optim import lr_scheduler
from torch.utils.data import Dataset,DataLoader
from torchvision import transforms, utils,models
import copy
import math
from skimage import io, transform
from sklearn.model_selection import train_test_split
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
dataset = pd.read_csv('/content/drive/MyDrive/8th Semester/MajorProject/Dataset/Memorability Scores/memorability-scores.csv')
dataset_train, dataset_validation = train_test_split(dataset, train_size=0.75, shuffle=True)

In [ ]:
dataset_train

,Filename,Image #,Hits,Misses,False Alarms,Correct Rejections,Hit Rate (HR),False Alarm Rate (FAR),Corrected Hit Rate
2040,Google_1_Cynthia Tejada_5_oval.jpg,2041,39,48,27,74,0.448,0.267,0.181
325,Google_1_Hazel Culp_17_oval.jpg,326,43,32,13,72,0.573,0.153,0.420
2115,Google_1_Danielle Wilbanks_5_oval.jpg,2116,32,32,23,51,0.500,0.311,0.189
911,Google_1_Carl Weston_19_oval.jpg,912,50,34,10,91,0.595,0.099,0.496
1478,Google_1_William Wallace_15_oval.jpg,1479,43,33,13,74,0.566,0.149,0.416
...,...,...,...,...,...,...,...,...,...
60,Google_1_Megan Springer_15_oval.jpg,61,51,39,12,93,0.567,0.114,0.452
1493,Google_1_Melissa Cushing_5_oval.jpg,1494,49,26,16,70,0.653,0.186,0.467
1287,Google_1_Jack Leslie_7_oval.jpg,1288,41,46,16,91,0.471,0.150,0.322
1763,Google_1_Hannah Ouellette_9_oval.jpg,1764,40,62,21,89,0.392,0.191,0.201


In [ ]:
class FaceDataset(Dataset):
      def __init__(self,dataframe,root_dir,transform=None):
        """
            Args:
                csv_file (string): Path to the csv file with annotations.
                root_dir (string): Directory with all the images.
                transform (callable, optional): Optional transform to be applied
                    on a sample.
        """
        self.data = dataframe
        #     self.data.rename(columns=columns,inplace=True)
    #     self.data.drop(self.data.columns[[1,2,3,4,5,6,8,9]] , axis=1,inplace=True)
        self.root_dir = root_dir
        self.transform = transform

      def __len__(self):
        return len(self.data)

      def __getitem__(self,idx):

        if torch.is_tensor(idx):
          idx = idx.tolist()

        image_name =  os.path.join(self.root_dir,self.data.iloc[idx,0])
        image = io.imread(image_name)
        #image = transformer(image)
        #image = resize(image, (227,227))
        mem_val = self.data.iloc[idx,8]
#     return_sample={}
        return_sample = {
              'image':image,
              'memorability_score':mem_val
        }
        if self.transform:
            return_sample = self.transform(return_sample)


        return return_sample

class Rescale(object):
    """Rescale the image in a sample to a given size.

    Args:
        output_size (tuple or int): Desired output size. If tuple, output is
            matched to output_size. If int, smaller of image edges is matched
            to output_size keeping aspect ratio the same.
    """

    def __init__(self, output_size):
        assert isinstance(output_size, (int, tuple))
        self.output_size = output_size

    def __call__(self, sample):

        image,mem_val = sample['image'], sample['memorability_score']

        h, w = image.shape[:2]

        # if isinstance(self.output_size, int):
        #     if h > w:
        #         new_h, new_w = self.output_size * h / w, self.output_size
        #     else:
        #         new_h, new_w = self.output_size, self.output_size * w / h
        # else:
        #     new_h, new_w = self.output_size

        # new_h, new_w = int(new_h), int(new_w)

        img = transform.resize(image, (self.output_size,self.output_size,3))
        return {'image': img, 'memorability_score': mem_val}
class ToTensor(object):
    """Convert ndarrays in sample to Tensors."""

    def __call__(self, sample):

        image, mem_val = sample['image'], sample['memorability_score']
#         print(type(torch.from_numpy(image)))
        # swap color axis because
        # numpy image: H x W x C
        # torch image: C X H X W
#         print(image.shape)

        image = image.transpose((2, 0, 1))

        return {'image': torch.from_numpy(image),
                'memorability_score': mem_val}
class Normalize(object):
    def __init__(self,mean,std):
        self.mean=mean
        self.std=std
    def __call__(self,sample):
        image, mem_val = sample["image"], sample["memorability_score"]
        normalized=  (image -self.mean) / self.std
        return {
            "image":normalized,
            "memorability_score" : mem_val
       }



In [ ]:
transformed_dataset_train = FaceDataset(dataset_train,root_dir="../Dataset/Memorability Scores/Face Images",
                                        transform=transforms.Compose([Rescale(227),ToTensor(),Normalize(0.5,0.5)]))

transformed_dataset_val= FaceDataset(dataset_validation,root_dir="../Dataset/Memorability Scores/Face Images",
                                        transform=transforms.Compose([Rescale(227),ToTensor(),Normalize(0.5,0.5)]))

train_dataloader=DataLoader(transformed_dataset_train,batch_size=4,shuffle=True)
validation_dataloader=DataLoader(transformed_dataset_val,batch_size=4,shuffle=True)

dataloaders={
    "train":train_dataloader,
    "val":validation_dataloader
}
dataset_sizes ={
    "train":len(dataset_train),
    "val":len(dataset_validation)
}

In [ ]:
len(transformed_dataset_train)

1666

In [ ]:
def getMSE(d1, d2):
        mse = 0.0
        for a,b in zip(d1, d2):
            mse += (a-b)**2
        return mse / len(d1)

def getRankCorrelation(predicted, gt=None):

        if gt is None:
            return "needed gt"
        gt = np.array(gt).tolist()
        predicted = np.array(predicted).squeeze().tolist()

        n = min(len(predicted), len(gt))
        if n < 2:
            return 0

        gt = gt[:n]
        predicted = predicted[:n]
        mse = getMSE(gt, predicted)

        def get_rank(list_a):
            rank_list = np.zeros(len(list_a))
            idxs = np.array(list_a).argsort()
            for rank, i in enumerate(idxs):
                rank_list[i] = rank

            return rank_list

        gt_rank = get_rank(gt)
        predicted_rank = get_rank(predicted)
        ssd = 0
        for i in range(len(predicted_rank)):
            ssd += (gt_rank[i] -  predicted_rank[i])**2

        rc = 1-(6*ssd/(n*n*n - n))


        return rc, mse

In [ ]:
pip install resmem

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 489.1/489.1 MB 1.3 MB/s eta 0:00:00


In [ ]:
from resmem import ResMem, transformer
resmem = ResMem(pretrained=True)
resmem.to(device)
resmem=copy.deepcopy(resmem)

/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [ ]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=5):
    since = time.time()
    running_loss_history = []
    val_running_loss_history=[]
    orignal_model=None
    best_model_wts = copy.deepcopy(model.state_dict())
    low_loss = np.inf

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0
            rank_corr = []

            # Iterate over data.
            for batched_data in tqdm(dataloaders[phase]):

                inputs=batched_data["image"]
                inputs = inputs.to(device)
                labels=batched_data["memorability_score"]
                labels=labels.view(-1,1).double()
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    loss = criterion(outputs, labels)
                    if phase == "val":
                      rc,mse = getRankCorrelation(outputs.squeeze().tolist(),labels.squeeze().tolist())
                      rank_corr.append(rc)
                    print("  batch loss:    ",loss.item())
                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            if phase=="train":
              running_loss_history.append(epoch_loss)
            else:
              val_running_loss_history.append(epoch_loss)


            print('{} Loss: {:.4f}'.format(
                phase, epoch_loss))


            # deep copy the model
            if phase == "val":
                rho =sum(rank_corr)/len(rank_corr)
                print("rank correlation,final",rho)
                if rho >=0.67:
                  print("rank correlation final",rho)
                  stop = True

            if phase == 'val' and epoch_loss < low_loss:
                low_loss = epoch_loss
                best_model_wts = copy.deepcopy(model.state_dict())
                print("saving best model....")
                torch.save(best_model_wts,"/content/drive/MyDrive/8th Semester/MajorProject/Dataset/CelebA/saved_models/ResMemNet2.pth")

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))

    # load best model weights
    original_model = copy.deepcopy(model)
    model.load_state_dict(best_model_wts)
    return model,original_model,running_loss_history,val_running_loss_history

In [ ]:
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(resmem.parameters(), lr = 0.0005, weight_decay = 0.0001)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=4, gamma=0.1)

In [ ]:
best_model,resmem,running_loss_history,val_running_loss_history = train_model(resmem,criterion,optimizer,exp_lr_scheduler,20)

Epoch 0/19
----------


  0%|          | 0/53 [00:00<?, ?it/s]

  batch loss:     0.195136674074681


  2%|▏         | 1/53 [00:38<33:20, 38.46s/it]

  batch loss:     0.10590159830507982


...

  batch loss:     0.026941444265660294


 94%|█████████▍| 17/18 [01:31<00:05,  5.33s/it]

  batch loss:     0.015467599856668266


100%|██████████| 18/18 [01:33<00:00,  5.20s/it]

  batch loss:     0.010537450441224769
val Loss: 0.0190
rank correlation,final 0.39641765044990845

Training complete in 162m 16s


In [ ]:
best_model.to(device)

ResMem(
  (features): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(
          

###Calculating memorability scores

In [ ]:
import numpy as np
from PIL import Image, ImageDraw
from pathlib import Path
import glob
import os
img_dir = ('../Cropped Images/Modified/Class_A_cropped')
img_files = os.listdir(img_dir)
len(img_files)

401

In [ ]:
resmem.load_state_dict(torch.load('../Dataset/CelebA/saved_models/ResMemNet.pth'))
resmem.eval()
predictions = []

#img_dir = ('../Dataset/trial_test/B2A/modified/cropped')
#img_files = os.listdir(img_dir)
tmp = np.arange(len(img_files))
x_test = []
for file in np.array(img_files)[tmp]:
    x_test.append(np.array(transformer(Image.open(img_dir +'/'+ file).convert('RGB'))))
x_test = np.array(x_test)
test_dataloader=DataLoader(x_test,batch_size=1,shuffle=False)

for img in tqdm(test_dataloader):

  prediction = resmem(img.view(-1, 3, 227, 227).to(device))
  predictions.append(prediction)


100%|██████████| 401/401 [00:06<00:00, 57.75it/s]


In [ ]:
array = []
for i in range(401):
  array.append(predictions[i].item())
array = np.array(array)
pd.DataFrame(array).to_csv('Modified_A.csv')